# Исследование надёжности заёмщиков


# Цель:

Основная цель проекта это разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

# Задачи:

- определить и заполнить пропущенные значения;

- заменить вещественный тип данных на целочисленный;

- удалить дубликаты;

- выделить леммы в значениях столбца с целями получения кредита;

- категоризовать данные.


# Аббревиатуры:

## Датасет *data.csv*:

- children — количество детей в семье

- days_employed — общий трудовой стаж в днях

- dob_years — возраст клиента в годах

- education — уровень образования клиента

- education_id — идентификатор уровня образования

- family_status — семейное положение

- family_status_id — идентификатор семейного положения

- gender — пол клиента

- income_type — тип занятости

- debt — имел ли задолженность по возврату кредитов

- total_income — ежемесячный доход

- purpose — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
bank = pd.read_csv("/datasets/data.csv")
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

При первом анализе, сразу же можно увидеть, что в столбце family_status используются разные наименование или "женат/замужем" или "гражданский брак". Такая же проблема часто встречается проблема в столбце purpuse. Еще могут возникнуть проблемы в столбце education, так как одни и те же слова начинаются или с маленькой или с большой буквы. Вообщем, таблицу сложно анализировать просто смотря на нее, нужно убрать все недочеты и постарать сделать таблицу более удобной для изучения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(bank["days_employed"].isnull().sum())
print(bank["dob_years"].isnull().sum())
print(bank["education_id"].isnull().sum())
print(bank["total_income"].isnull().sum())
bank.dropna(inplace=True)




2174
0
0
2174


In [3]:
bank["gender"].unique()



array(['F', 'M', 'XNA'], dtype=object)

### Вывод

После анализа всех столбцов, я обнаружил пропуски пропуски в столбцах "days_employed" и "total_income". Интересно, что оба пропуска появлялись в одной и той же строке. Первая мысль у меня была, заполнить пропуски средними числами, но тут подключались другие столбцы и показывали, что люди очень разные, и высчитать среднее для них всех будет не точно и проблемотично. Я решил от них избавиться, так как в строках где встречаются пропуски имеют мало общего между друг другом, и на статистику не сильно повлияют. 
Думаю, что эти пропуски появились не из за какой то ошибки, а видимо в каком то отделении систематично не учитывались эти параметры, так как как пропуски везде в одних и тех же строках.

### Замена типа данных

In [4]:
print(bank.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB
None


In [5]:
bank["days_employed"]=bank["days_employed"].astype("int")
bank["total_income"]=bank["total_income"].astype("int")

In [6]:
print(bank.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB
None


### Вывод

Что бы  точно убедиться какие из столбцов являются вещественными типами данных, я вызвал функцию info(). Далее я использовал функцию astype(), что бы заменить тип данных.

### Обработка дубликатов

In [7]:
bank["education"] = bank["education"].str.lower()
print(bank["education"].unique())

print(bank["income_type"].unique())
bank["children"] = bank["children"].replace(-1, 1)
bank["children"] = bank["children"].replace(20, 2)
bank["children"].unique()
bank["family_status"] = bank["family_status"].replace("гражданский брак", "женат / замужем")
bank["family_status"].unique()


['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный' 'студент'
 'предприниматель' 'в декрете']


array(['женат / замужем', 'вдовец / вдова', 'в разводе',
       'Не женат / не замужем'], dtype=object)

### Вывод

Проанализировав столбцы, я нашел дубликаты разные их типы. Самые очеведные оказались в столбце "education", где одно и тоже слово могло быть написано или с заглавной буквой, или все были заглавными буквами, или все маленькие. Я решил используя функцию string.lower(), сделать все слова в столбце используя только маленькие быквы. Я дпредпологаю, что они появились, так как в ручную вбивались эти данные, и вписывали как хотели. 

### Лемматизация

In [8]:
bank["purpose"].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'получение образования',
       'сделка с подержанным автомобилем', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее образование',
       'покупка жилья для сдачи

In [9]:
from pymystem3 import Mystem
m = Mystem()
def purpose_change(row):
    purpose = " ".join(m.lemmatize(row["purpose"]))
    if 'автом' in purpose:
        return 'автомобиль'
    elif 'свадь' in purpose:
        return  'свадьба'
    elif 'образов' in purpose:
        return 'образование'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'недвижимость'
    else:
        return row['purpose']

bank['purpose_new']= bank.apply(purpose_change, axis=1)
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,женат / замужем,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Что бы осуществоить лемматизацию столбца purpose, я вызвал функцию unique() и изучил необходимые корни. Далее я понял, что надо будет создавать новый столбец. При помощи If, Elif, Else я создал условие которое бы обрабатывало корень слова со столбца purpose выводило бы мною заданое слово в новый столбец.

### Категоризация данных

In [10]:
bank["total_income"].max()

2265604

In [11]:
bank["total_income"].min()

20667

In [12]:
bank["total_income"].mean()

167421.80171567362

In [13]:

bank["total_income"].describe()

count    1.935100e+04
mean     1.674218e+05
std      1.029716e+05
min      2.066700e+04
25%      1.030530e+05
50%      1.450170e+05
75%      2.034345e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [14]:

def level_income(row):
    purpose = row["purpose_new"]
    income = row["total_income"]
    debt = row["debt"]
    if purpose == "автомобиль" or "образование" or "свадьба" or "недвижимость":
        if income < 70000:
            if debt == 0:
                return "Средний уровень доверия"
    if purpose == "автомобиль" or "образование" or "свадьба" or "недвижимость":
        if 70000 <= income < 200000:
            if debt == 0:
                return "Высокий уровень доверия"
    if purpose == "автомобиль" or "образование" or "свадьба" or "недвижимость":
        if 200000 <= income:
            if debt == 0:
                return "Очень высокий уровень доверия"
    if purpose == "автомобиль" or "образование" or "свадьба" or "недвижимость":
        if income < 70000:
            if debt == 1:
                return "Низкий уровень доверия"
    if purpose == "автомобиль" or "образование" or "свадьба" or "недвижимость":
        if 70000 <= income < 200000:
            if debt == 1:
                return "Средний уровень доверия"
    if purpose == "автомобиль" or "образование" or "свадьба" or "недвижимость":
        if 200000 <= income:
            if debt == 1:
                return "Высокий уровень доверия"
bank["level_income"]= bank.apply(level_income, axis = 1)
bank["level_income"].value_counts()


Высокий уровень доверия          12057
Очень высокий уровень доверия     4708
Средний уровень доверия           2485
Низкий уровень доверия             101
Name: level_income, dtype: int64

In [15]:

def wealth(row):
    income = row["total_income"]
    if income < 70000:
            return "Низкий уровень дохода"
    if 70000 <= income < 200000:
            return "Средний уровень дохода"
    if 200000 <= income:
            return "Высокий уровень дохода"
bank["wealth"]= bank.apply(wealth, axis = 1)
bank["wealth"].value_counts()


Средний уровень дохода    12811
Высокий уровень дохода     5066
Низкий уровень дохода      1474
Name: wealth, dtype: int64

### Вывод

Что бы помочь банку наглядней увидеть кому лучше давать кредит, а кому менее желатель, я создал 4 категории основываясь на зарпалте, история возврата кредита, и зачем людям нужен кредит. Если у человека высокий доход и нету проблем с возвратом кредита, то этим людям желательно выдавать кредит. Если наоборот, то нужно более внимательнее отнестись к выдаче кредитов этим людям.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
data_pivot = bank.pivot_table(index=["children"], values = "debt", aggfunc=["count","sum","mean"])
data_pivot

,count,sum,mean
,debt,debt,debt
children,,,
0,12710,952,0.074902
1,4387,409,0.093230
2,1918,185,0.096455
3,294,22,0.074830
4,34,3,0.088235
5,8,0,0.000000


### Вывод

Новый вывод: я добавил два дополнительных столбца, это mean и count, что бы увидеть не только количество людей, но и среднее, для вывода более точных выводов. После добавления, мои ранее выводы оказались ложными, о том, что люди без детей хуже всех отдают кредиты вовремя. Оказалось, что по количеству это правда, но это так потому что, группа этих людей впринципе оказалась самая большая. А в среднем, хуже всех кредиты отдают люди имея двух детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
data_pivot = bank.pivot_table(index=["family_status"], values = "debt", aggfunc=["count","sum","mean"])
data_pivot

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2525,254,0.100594
в разводе,1083,76,0.070175
вдовец / вдова,865,56,0.064740
женат / замужем,14878,1185,0.079648


### Вывод

Новый вывод: опять же мой предыдущий вывод оказался ложным, в категории женат/замужем больше всех людей, и из за этого у них в по количеству больше всех должников. В среднем, хуже всех отдают люди которые в категории не женат/не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
data_pivot = bank.pivot_table(index=["wealth"], values = "debt", aggfunc=["count","sum","mean"])
data_pivot

,count,sum,mean
,debt,debt,debt
wealth,,,
Высокий уровень дохода,5066,358,0.070667
Низкий уровень дохода,1474,101,0.068521
Средний уровень дохода,12811,1112,0.086800


### Вывод

Здесь можно наглядо увидеть, что лучше всех отдают долги люди с низким уровнем дохода, и после них люди с высоким уровнем дохода. Хуже всех отдают со средним уровнем дохода, и разрыв между другими категориями очень значительный.
Новый вывод: мой предыдущий вывод оказался правдивым, но теперь я могу наглядно увидеть, что даже в среднем люди в со средним уровнем дохода хуже отдают долги. Но я был не прав в одном, что разрыв очень значительный, столбец "mean" наглядно показывает, что разраыв не такой уж и значительный, а разница в количестве людей очень высокая между средним уровнем дохода, и других категорий.

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
data_pivot = bank.pivot_table(index=["purpose_new"], values = "debt", aggfunc=["count","sum","mean"])
data_pivot

,count,sum,mean
,debt,debt,debt
purpose_new,,,
автомобиль,3897,367,0.094175
недвижимость,9758,715,0.073273
образование,3597,331,0.092021
свадьба,2099,158,0.075274


### Вывод

Лучше всего отдают кредиты когда берут под свадьбу и на образование. Хуже всего отдают при покупке недвижимости. 
Новый вывод: Оказалось, что я вообще был не прав. Хуже всех отдают кредиты люди покупающие автомобили и на берущих на образование. 

### Шаг 4. Общий вывод

В общем, хочется сделать портрет человека, который лучше всего будет возвращает кредит в срок, и самого худшего. Лучший, это человек с большим количеством детей, нету второй половинки, с низким уровнем дохода, и берет деньги в кредит на свадьбу. Худший, это человек не имеющий детей, или их мало, женат/замужем, со средним уровнем дохода, и берет он кредит на покупку недвижимости.
Новый общий вывод: после изменения в коде, хочу еще раз описать "идеального" клиента, и наименее надежного. Идеальный: вдовец или вдова, имеет 5 и более детей, низкий уровень дохода, и этот человек берет кредит для покупки недвижимости. Наиболее опасный: имеет двух детей, средний уровень дохода, не женат или не замужем и берет кредит на покупку автомобиля. После проделанной работы, можно с уверенностью, что при собирании данных происходит много ошибок, такие как пропуски, непонятные значение, отрицательные числа без объяснения, и из-за этого приходиться работать не с полными данными и даже удалять некоторые строки. Это может повлиять на конечный вывод и принести убытки компании. Все это происходит, из-за того, что сотрудники или клиенты вписывают данные вручную, и случайно делают ошибки. Что бы избежать в дальнейшем таких проблем, рекомендую создать опросник, где нельзя было бы вписывать ответы в формате как понимают люди, наглядно это было видно в столбце "purpose" где люди писали одни и те же цели по-разному. Нужно создать уже заготовленные ответы, например в столбце пол должно быть только 2 варианта, или мужчина или женщина. Если, например в столбце нет варианта, который подходит клиенту, только в этом случае можно добавлять строку "Свой вариант".
